In [2]:
import pandas as pd

dataset = pd.read_csv('raportVanzari.csv', encoding='utf-8', encoding_errors='ignore')
dataset['MODEL'] = dataset['MODEL'].str.replace(' ','', regex = False)
grupare_modele = {model: dataset[dataset['MODEL'] == model] for model in dataset['MODEL'].unique()}

print(grupare_modele)

{'DUSTER':      NR. CRT.                VIN DATA INTRARE VO TIP AUTOVEHICUL  MARCA  \
0           1  VF1HJD40967321860      20/09/2024      AUTOTURISM  DACIA   
1           2  VF1HJD40X67425287      02/10/2024      AUTOTURISM  DACIA   
3           4  VF1HJD40X67418940      19/09/2024      AUTOTURISM  DACIA   
4           5  VF1HJD40467321717      02/10/2024      AUTOTURISM  DACIA   
6           7  VF1HJD40767321856      03/10/2024      AUTOTURISM  DACIA   
..        ...                ...             ...             ...    ...   
697        18  VF1HJD20367813156      17/07/2025      AUTOTURISM  DACIA   
715         3  VF1HJD40367329226      22/04/2025      AUTOTURISM  DACIA   
722        10  VF1HJD40767814291      11/06/2025      AUTOTURISM  DACIA   
723        11  VF1HJD20867813153      17/07/2025      AUTOTURISM  DACIA   
726        14  VF1HJD40467418495      04/04/2025      AUTOTURISM  DACIA   

      MODEL     VERSIUNE         CULOARE  AN FABRICATIE        KM  ...  \
0    DUSTER  S

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xlsxwriter

def main():
    # Initializarea unor vectori pentru stocarea mediilor fiecarui model/an de fabricatie/versiune pentru a observa diferentele dintre date
    medii_vanzare_model = []
    std_vanzare_model = []
    lista_modele = []

    medii_vanzare_an_fabricatie = []
    std_vanzare_an_fabricatie = []
    lista_an_fabricatie = []

    medii_vanzare_versiune = []
    std_vanzare_versiune = []
    lista_versiune = []

    # --- Initializare Excel ---
    workbook = xlsxwriter.Workbook('Rezultate - Excel.xlsx', {'nan_inf_to_errors': True})
    worksheet = workbook.add_worksheet('Rezultate Vanzari')

    # Headers for grouped results (A1:D1) and single-column results (F1:K1)
    headers = [
        # Gruparea Model/Echipare/An
        'Model', 'Echipare', 'An de fabricatie', 'Medie Vanzare Combi', 'Abatere Std Combi',
        # Gruparea Model
        'Modele (Total)', 'Medie/model (Total)', 'Abatere standard/model (Total)',
        # Gruparea An de Fabricatie
        'An de fabricatie (Total)', 'Medie/An (Total)', 'Abatere standard/An (Total)',
        # Gruparea Versiune
        'Echipare (Total)', 'Medie/Echipare (Total)', 'Abatere standard/Echipare (Total)'
    ]
    worksheet.write_row('A1', headers)

    # --- Incarcare Date ---
    try:
        dataset = pd.read_csv('raportVanzari.csv', encoding='utf-8', encoding_errors='ignore')
    except FileNotFoundError:
        print("Eroare: Fisierul 'raportVanzari.csv' nu a fost gasit.")
        return

    if dataset.empty:
        print("Excelul de date este gol, eroare la incarcare.")
        return
    print("\nVerificarea incarcarii datelor...", dataset.head(2))

    # --- Curatare si Conversie de Date ---
    dataset['MODEL'] = dataset['MODEL'].str.replace(' ', '', regex=False)

    # Curatarea coloanei 'VANZARE'
    dataset['VANZARE'] = dataset['VANZARE'].astype(str).str.replace(',', '', regex=False).str.replace('?', '', regex=False).str.replace('lei', '', regex=False).str.strip()
    # Conversia la float
    try:
        dataset['VANZARE'] = pd.to_numeric(dataset['VANZARE'], errors='coerce')
    except Exception as e:
        print(f"Eroare la conversia coloanei 'VANZARE' la float: {e}")
        return

    # Ștergere rânduri unde 'VANZARE' este NaN (Corectat)
    dataset_initial_rows = len(dataset)
    dataset = dataset.dropna(subset=['VANZARE'])
    print(f"Au fost eliminate {dataset_initial_rows - len(dataset)} rânduri cu valori lipsă în 'VANZARE'.")

    # Verificare finală
    nr_valori_lipsa = dataset['VANZARE'].isnull().sum()
    if nr_valori_lipsa > 0:
        print("\nEroare: Au rămas valori NaN în 'VANZARE' după dropna!")
        return

    # --- 1. Gruparea Principală (Model, Versiune, An Fabricatie) ---
    print("\nCalculul mediilor pe combinatia MODEL / VERSIUNE / AN FABRICATIE...")
    rezultate_combinate = dataset.groupby(['MODEL', 'VERSIUNE', 'AN FABRICATIE'])['VANZARE'].agg(['mean', 'std']).reset_index()

    current_row = 1
    for index, row in rezultate_combinate.iterrows():
        worksheet.write(current_row, 0, row['MODEL'])
        worksheet.write(current_row, 1, row['VERSIUNE'])
        worksheet.write(current_row, 2, row['AN FABRICATIE'])
        worksheet.write(current_row, 3, row['mean']) # Medie Vanzare Combi
        worksheet.write(current_row, 4, row['std'])  # Abatere Std Combi
        current_row += 1

    # --- 2. Gruparea Individuală (Modele) ---
    print("\nCalculul mediilor pe MODEL (Total)...")
    grupare_modele = dataset.groupby('MODEL')['VANZARE'].agg(['mean', 'std']).reset_index()
    for index, row in grupare_modele.iterrows():
        worksheet.write(index + 1, 5, row['MODEL'])
        worksheet.write(index + 1, 6, row['mean'])
        worksheet.write(index + 1, 7, row['std'])
        print(f"Pentru modelul {row['MODEL']}, media de vanzare este: {row['mean']:.2f}.")

    # --- 3. Gruparea Individuală (An Fabricatie) ---
    print("\nCalculul mediilor pe AN FABRICATIE (Total)...")
    grupare_an_fabricatie = dataset.groupby('AN FABRICATIE')['VANZARE'].agg(['mean', 'std']).reset_index()
    for index, row in grupare_an_fabricatie.iterrows():
        worksheet.write(index + 1, 8, row['AN FABRICATIE'])
        worksheet.write(index + 1, 9, row['mean'])
        worksheet.write(index + 1, 10, row['std'])
        print(f"Pentru anul de fabricatie {row['AN FABRICATIE']}, media de vanzare este: {row['mean']:.2f}.")

    # --- 4. Gruparea Individuală (Versiune/Echipare) ---
    print("\nCalculul mediilor pe VERSIUNE/ECHIPARE (Total)...")
    grupare_versiune = dataset.groupby('VERSIUNE')['VANZARE'].agg(['mean', 'std']).reset_index()
    for index, row in grupare_versiune.iterrows():
        worksheet.write(index + 1, 11, row['VERSIUNE'])
        worksheet.write(index + 1, 12, row['mean'])
        worksheet.write(index + 1, 13, row['std'])
        print(f"Pentru varianta de echipare {row['VERSIUNE']}, media de vanzare este: {row['mean']:.2f}.")

    # Nota: Gruparile pe Model/An/Echipare (total) sunt scrise in coloane separate (F, I, L)
    # incepand cu randul 2, folosind indexul.

    workbook.close()
    print("\n✅ Rezultatele au fost scrise cu succes în 'ExcelRezultateVanzari.xlsx'.")

if __name__ == "__main__":
    main()


Verificarea incarcarii datelor...    NR. CRT.                VIN DATA INTRARE VO TIP AUTOVEHICUL  MARCA   MODEL  \
0         1  VF1HJD40967321860      20/09/2024      AUTOTURISM  DACIA  DUSTER   
1         2  VF1HJD40X67425287      02/10/2024      AUTOTURISM  DACIA  DUSTER   

      VERSIUNE        CULOARE  AN FABRICATIE        KM  ... Rate facturate  \
0  SL BLUELINE     GRI COMETE           2021   97038.0  ...             37   
1  SL BLUELINE  GRIS HIGHLAND           2021  130538.0  ...             39   

   Perioada contract  CIV IESIRE ROL LIVRATE LA CLIENT DATA OFERTA ANGAJATI  \
0                 42  CIV        NaN               NaN                    -   
1                 42  CIV        NaN               NaN                    -   

  Data evaluare.1 Zile stoc / data evaluare Zile stoc / data intrare  \
0      02.10.2024                   #VALUE!                      409   
1      02.10.2024                   #VALUE!                      397   

  Viteza rotatie  
0           

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xlsxwriter
import seaborn as sns

def main():

    workbook = xlsxwriter.Workbook('Rezultate - Excel.xlsx', {'nan_inf_to_errors': True})
    worksheet =  workbook.add_worksheet('Grupare - Medii de vanzare')
    headers = [
        'MARCA', 'MODEL', 'VERSIUNE', 'AN FABRICATIE','CAPACITATE CILINDRICA', 'CULOARE', 'PUTERE','MEDIE DE VANZARE', 'COUNT'
    ]
    worksheet.write_row('A1', headers)
    dataset = pd.read_csv('raportVanzari.csv', encoding = 'utf-8', encoding_errors = 'ignore')
    if dataset.empty:
        print("Excelul de date este gol, eroare la incarcare.")
        return
    data_check = dataset.head()
    print("\nVerificarea incarcarii datelor...", data_check)

     #Curatare de date, coloana cu vanzari
    dataset['MODEL'] = dataset['MODEL'].str.replace(' ','', regex = False)
    dataset['VANZARE'] = dataset['VANZARE'].str.replace(',','', regex=False).str.replace('?','', regex=False).str.replace('lei', '', regex = False).str.strip().astype(float)


    dataset = dataset.dropna(subset = ['VANZARE'])
    nr_valori_lipsa = dataset['VANZARE'].isnull().sum()
    if nr_valori_lipsa > 0:
        print("\nEroare la stergerea coloanelor goale.")
        return
    grupare_marca = {marca: dataset[dataset['MARCA']==marca] for marca in dataset["MARCA"].unique()}

    current_row = 1
    for marca, df_marca in grupare_marca.items():

         # Grupare pe cele 6 atribute in cadrul marcii curente si calcul 'mean' si 'count'
         rezultate_combinate = df_marca.groupby([
             'MODEL', 'VERSIUNE', 'AN FABRICATIE','CAPACITATE CILINDRICA', 'CULOARE', 'PUTERE'
         ])['VANZARE'].agg(['mean', 'count']).reset_index()
         worksheet.write(current_row, 0, marca)

         for index, row in rezultate_combinate.iterrows():
            worksheet.write(current_row, 1, row['MODEL'])
            worksheet.write(current_row, 2, row['VERSIUNE'])
            worksheet.write(current_row, 3, row['AN FABRICATIE'])
            worksheet.write(current_row, 4, row['CAPACITATE CILINDRICA'])
            worksheet.write(current_row, 5, row['CULOARE'])
            worksheet.write(current_row, 6, row['PUTERE'])
            worksheet.write(current_row, 7, row['mean'])
            worksheet.write(current_row, 8, row['count'])
            current_row += 1


    workbook.close()

if(__name__ == '__main__'):
    main()









Verificarea incarcarii datelor...    NR. CRT.                VIN DATA INTRARE VO TIP AUTOVEHICUL    MARCA  \
0         1  VF1HJD40967321860      20/09/2024      AUTOTURISM    DACIA   
1         2  VF1HJD40X67425287      02/10/2024      AUTOTURISM    DACIA   
2         3  VF1RJL000UC347501      31/05/2024      AUTOTURISM  RENAULT   
3         4  VF1HJD40X67418940      19/09/2024      AUTOTURISM    DACIA   
4         5  VF1HJD40467321717      02/10/2024      AUTOTURISM    DACIA   

    MODEL        VERSIUNE         CULOARE  AN FABRICATIE        KM  ...  \
0  DUSTER     SL BLUELINE      GRI COMETE           2021   97038.0  ...   
1  DUSTER     SL BLUELINE   GRIS HIGHLAND           2021  130538.0  ...   
2  ARKANA  INTENS E-TECH   BLACK METALLIC           2022   23715.0  ...   
3  DUSTER        PRESTIGE   GRIS HIGHLAND           2021   69979.0  ...   
4  DUSTER        PRESTIGE   GRIS HIGHLAND           2021   22983.0  ...   

  Rate facturate  Perioada contract  CIV IESIRE ROL LIVRATE LA 

In [1]:
import pandas as pd
import xlsxwriter
import numpy as np


def main():

    dataset = pd.read_csv('raportVanzari.csv', encoding = 'utf-8', encoding_errors = 'ignore')
    if dataset.empty:
        print("Excelul de date este gol, eroare la incarcare.")
        return
    data_check = dataset.head()


    #Curatare de date, coloana cu vanzari
    dataset['MODEL'] = dataset['MODEL'].str.replace(' ','', regex = False)
    dataset['VANZARE'] = dataset['VANZARE'].str.replace(',','', regex=False).str.replace('?','', regex=False).str.replace('lei', '', regex = False).str.strip().astype(float)


    dataset = dataset.dropna(subset = ['VANZARE'])
    nr_valori_lipsa = dataset['VANZARE'].isnull().sum()
    if nr_valori_lipsa > 0:
        print("\nEroare la stergerea coloanelor goale.")
        return

    mask = dataset['CAPACITATE CILINDRICA'].isna()
    modele_valori_lipsa = dataset['MODEL'][mask]
    print(modele_valori_lipsa)

    print("\n")
    coloane = dataset.columns
    print(coloane)

if(__name__ == '__main__'):
    main()

227     RAFALE
339      ARIYA
340      ARIYA
341      ARIYA
342      ARIYA
343      ARIYA
344      ARIYA
345      ARIYA
346      ARIYA
347      ARIYA
348      ARIYA
349      ARIYA
350      ARIYA
351      ARIYA
352      ARIYA
353      ARIYA
354      ARIYA
355      ARIYA
356      ARIYA
357      ARIYA
358    X-Trail
359    X-Trail
360    X-Trail
361    X-Trail
362    X-Trail
363    X-Trail
364    X-Trail
365    X-Trail
366    X-Trail
367    X-Trail
368    X-Trail
369    X-Trail
370    X-Trail
371    X-Trail
372    X-Trail
373    X-Trail
374    X-Trail
375    X-Trail
Name: MODEL, dtype: object


Index(['NR. CRT.', 'VIN', 'DATA INTRARE VO', 'TIP AUTOVEHICUL', 'MARCA',
       'MODEL', 'VERSIUNE', 'CULOARE', 'AN FABRICATIE', 'KM',
       'CAPACITATE CILINDRICA', 'PUTERE', 'DATA PRIMEI INMATRICULARI',
       'FURNIZOR', 'PROVENIENTA', 'Tip', 'LOCATIA', 'PRET CUMPARARE HT LEI',
       'PRET EVALUARE TTC EURO', 'Data validare pret evaluare', 'REZERVARI',
       'NR NIR', 'OBSERVATII', 'Serie car